In [2]:
import cv2
import numpy as np

In [ ]:
def blur(img):
    return cv2.GaussianBlur(img,(0,0),1.5)

def top_hat(img):
    kernel=np.ones((5,5),dtype=np.int16)
    return cv2.dilate(cv2.erode(img,kernel,iterations=3),kernel,iterations=2)

def pre_process(img):
    return cv2.Canny(top_hat(blur(img)),100,255)

if __name__=='__main__':
    # ground truth
    GT=[920,970,980,1080,2080,2290,2190,2580,2730,2230,
        2530,2030,1530,2130,2030,2430,2330,1340,850,840,
        1240,1730,1680,1180,1170,1170,1660,1160,1110,1110]
    
    predict=[]
    
    IMG_CNT=33
    FILEPATH='./Coins/'
    
    for i in range(4,IMG_CNT+1):
        #print('img: #',i)
        src=cv2.imread(FILEPATH+str(i)+".jpg",cv2.IMREAD_COLOR)
        src=cv2.resize(src,dsize=(0,0),fx=0.25,fy=0.25,interpolation=cv2.INTER_LINEAR)
        
        gray=cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
        cv2.imshow("gray",gray)
        
        # preprocess - blur, tophat, canny
        edge=pre_process(gray)
        cv2.imshow('edge',edge)
        
        # HoughCircle
        circles=cv2.HoughCircles(edge,cv2.HOUGH_GRADIENT,1,60,param1=50,param2=28.5,minRadius=5,maxRadius=100)
        if circles is None: # if not detected circle
            print("detected circles :", 0)
            cv2.waitKey(0)
            continue
        
        # sort detected circles by increasing order of radius
        float_circles=circles[0].copy()
        float_circles=np.array(sorted(float_circles,key=lambda x:x[2]))
        
        total=0

        # draw circles
        plot_img=src.copy()
        hsv_img=src.copy()
        for j,circle in enumerate(float_circles):
            plot_circle=np.uint16(np.around(circle))
            #print(circle[2])
            cx,cy,rad=plot_circle[0],plot_circle[1],plot_circle[2]
            cv2.circle(plot_img,(cx,cy),rad,(0,0,255),2) # draw the boundary of circle
            cv2.circle(plot_img,(cx,cy),2,(0,0,255),2) # draw the center of the circle

            ### Rule-based
            x1,x2,y1,y2=int(cx-rad),int(cx+rad),int(cy-rad),int(cy+rad)
            
            crop=hsv_img[y1:y2,x1:x2,:]
            crop_h,crop_w=crop.shape[:2]
            
            mask=np.zeros_like(crop, np.uint8)
            cv2.circle(mask,(crop_w//2, crop_h//2),rad,(255,255,255),-1)
            
            crop=cv2.bitwise_and(crop,mask)
            
            hsv = cv2.cvtColor(crop,cv2.COLOR_BGR2HSV)
            hue, sat, val= cv2.split(hsv)

            mask = np.zeros((crop_h, crop_w), np.uint8)
            cv2.circle(mask, (crop_w//2, crop_h//2), rad, 255, -1)
            #mean_of_hue = cv2.mean(hue, mask)[0]
            mean_of_sat = cv2.mean(sat, mask)[0]
            #mean_of_val = cv2.mean(val, mask)[0]
            
            # rule-based
            won=-1
            if mean_of_sat < 28.0: # 1000
                won=1000
                
            elif mean_of_sat > 98: # 10won
                won = 10
            else:
                if circle[2]<=45.0: # 50won
                    won=50
                elif circle[2]>=45.39: # 100won
                    won=100
            
            if circle[2]>=50.0: # 500won
                won=500
            
            #print(j, won, circle[2], mean_of_sat)
            total+=won
            
            cv2.putText(plot_img,str(j)+'/'+str(won),(cx-15,cy),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2,cv2.LINE_AA)
        
        predict.append(total)
        cv2.putText(plot_img,'total:'+str(total),(200,200),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2,cv2.LINE_AA)
        
        cv2.imshow('plot_img',plot_img)
        cv2.imwrite(FILEPATH+str(i)+'_detected.jpg',plot_img)
        key=cv2.waitKey(0)
        if key==ord('q'):
            break

    cv2.destroyAllWindows()
    
    err=0
    for i,zipped in enumerate(zip(GT,predict)):
        if zipped[0]!=zipped[1]:
            err+=1
            print(i+4,'.jpg was wrong')
    print('\nacc:',(30-err)/30)